In [2]:
import numpy as np
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import optimizers
import time
import pylab
import matplotlib.pyplot as plt
import pickle
import csv
import cv2

from sklearn.datasets import load_digits
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import model_selection
#from sklearn.model_selection import GridSearchCV

In [2]:
gpu_flag = 0

if gpu_flag >= 0:
    cuda.check_cuda_available()
xp = cuda.cupy if gpu_flag >= 0 else np


with open('/home/dl-box/Liver/sotsuron_train/new1002mix_NC_classificate.pkl','rb') as f1:
    dataset1 = pickle.load(f1)
    
with open('/home/dl-box/Liver/sotsuron_data/test_data/Meta504_NC_classificate.pkl','rb') as f2:
    dataset2 = pickle.load(f2)

In [52]:

train_set_x, train_set_y = dataset1[0]
test_set_x, test_set_y = dataset2[0]

train_set_x = np.array(train_set_x).astype(xp.float32)
train_set_y = np.array(train_set_y).astype(xp.int32)
test_set_x = np.array(test_set_x).astype(xp.float32)
test_set_y = np.array(test_set_y).astype(xp.int32)

x_train = train_set_x.reshape(train_set_x.shape[0],-1)
y_train = train_set_y

x_test = test_set_x.reshape(test_set_x.shape[0],-1)
y_test = test_set_y

N = x_train.shape[0]
N_test = y_test.size

x_train = x_train.reshape((len(x_train), 1, 29, 29))
x_test = x_test.reshape((len(x_test), 1, 29, 29))


#feature extraction
class LeNet(chainer.Chain):
    def __init__(self):
        super(LeNet,self).__init__(
        conv1=F.Convolution2D(1, 32, 3),   
        conv2=F.Convolution2D(32, 64, 3), 
        conv3=F.Convolution2D(64, 64, 2),
        conv4=F.Convolution2D(64, 128, 2),
        conv5=F.Convolution2D(128, 128, 2),
        conv6=F.Convolution2D(128, 128, 2),
        conv7=F.Convolution2D(128, 256, 2),
        conv8=F.Convolution2D(256, 256, 2),
        conv9=F.Convolution2D(256, 256, 2),
        conv10=F.Convolution2D(256, 512, 2),
        conv11=F.Convolution2D(512, 512, 2),
        conv12=F.Convolution2D(512, 512, 2),
            l1=F.Linear(2048, 256),            
            l2=F.Linear(256, 6)#classification
        )
        
    def __call__(self,x_data,train = True):
        #x = chainer.Variable(x_data)
        x = chainer.Variable(x_data.astype(np.float32))
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.relu(self.conv5(h))
        h = F.relu(self.conv6(h))
        h = F.relu(self.conv7(h))
        h = F.relu(self.conv8(h))
        h = F.relu(self.conv9(h))
        h = F.relu(self.conv10(h))
        h = F.relu(self.conv11(h))
        h = F.relu(self.conv12(h))
        h = F.dropout(F.relu(self.l1(h)), train= train)
        y = self.l2(h)
        y = F.relu(y)
        return y

        
    #deside future accuracy
    def forward_Train(self, x_data,y_data):
        label_list = np.empty((0,6),int)
        
        y = self(x_data ,True)
        t = chainer.Variable(y_data)
        
        #tt = np.array([t])
        
        print type(y.data)
        #print y.data
        print y.data.shape
        print type(t)
        #print tt.shape
        
        label_list = np.array([ y.data[i] for i in range(0,y.data.shape[0])])
        label_list = label_list.reshape(label_list.shape[0],-1)
        #train_set_x = train_set_x.reshape(train_set_x.shape[0],-1)
        #label_list = np.array([label_list])
        #print label_list
        #print type(label_list)
        print label_list.shape
        
        
        parameters = {'kernel':['rbf'], 
              'C':[6],
              'gamma':[1e-2]}
        clf = GridSearchCV(SVC(), parameters)
        clf.fit(label_list, t.data)
        svmclf = clf.best_estimator_
        svmclf.fit(label_list, t.data)
        
        return F.softmax_cross_entropy(y, t), F.accuracy(y,t)
    
    def forward_Test(self, x_data, y_data):
        t = chainer.Variable(y_data)
        y = self(x_data)
        return F.accuracy(y, t)   
        
    def forward_label(self, x_data,y_data):
        label_list2 = []
        
        #label_list = np.empty((0,6),int)
        t = chainer.Variable(y_data)
        y = self(x_data)
        
        
        label_list2 = np.append(label_list, np.array([y.data[0]]), axis=0)
        
        y_testSVM = svmclf.predict(label_list2)
        print_cmx(t, y_testSVM)
        
        print(classification_report(t, y_testSVM))
        print("Accuracy: {0}".format(accuracy_score(t, y_testSVM)))

        #return label_list2

In [53]:
batchsize = 1000
n_epoch = 5

model = LeNet()
optimizer = optimizers.Adam()
optimizer.setup(model)

if gpu_flag >= 0:
    cuda.get_device(gpu_flag).use()
    model.to_gpu()
    
# ---Train---
for epoch in range(1, n_epoch + 1):
    print "epoch: %d" % epoch

    perm = np.random.permutation(N)
    for i in range(0, N, batchsize):
        x_batch = xp.asarray(x_train[perm[i:i + batchsize]],dtype=np.float32)
        y_batch = xp.asarray(y_train[perm[i:i + batchsize]],dtype=np.int32)
        
        model.zerograds()
        loss,train_acc = model.forward_Train(x_batch, y_batch)
        loss.backward()
        optimizer.update()
        

epoch: 1
<type 'cupy.core.core.ndarray'>
(1000, 6)
<class 'chainer.variable.Variable'>
(1000, 6)


ValueError: object __array__ method not producing an array

In [46]:
#---Test(simple 6 classes clasificate)---

start_time = time.clock()
min_result = 1.000
acc_label2 = []
max_acc=0.0

    
sum_accuracy = 0   
for i in range(0, N_test, batchsize):
    x_batch = xp.asarray(x_test[i:i + batchsize])
    y_batch = xp.asarray(y_test[i:i + batchsize])
    acc = model.forward_Test(x_batch, y_batch)
    sum_accuracy += float(acc.data) * len(y_batch)     
test_result = sum_accuracy / N_test


min_label = []
acc_label = np.empty((0,6),int)
for i in range(0, N_test, batchsize):
    x_batch = xp.asarray(x_test[i:i + batchsize])
    y_batch = xp.asarray(y_test[i:i + batchsize])
    model.forward_label(x_batch,y_batch)
    
    """
    acc = np.array([model.forward_label(x_batch,y_batch)])
    acc_label = np.append(acc_label, np.array([acc[0]]), axis=0)
acc_label2 = sum(acc_label)/N_test

print("---probability---")
print "Liver : %f" %(acc_label2[0])
print "Cyst  : %f" %(acc_label2[1])
print "FNH   : %f" %(acc_label2[2])
print "CCC   : %f" %(acc_label2[3])
print "HCC   : %f" %(acc_label2[4])
print "Meta  : %f" %(acc_label2[5])
    
end_time = time.clock()
#print end_time - start_time
"""

<type 'cupy.core.core.ndarray'>
(1000, 6)
<type 'cupy.core.core.ndarray'>
(1000, 6)
<type 'cupy.core.core.ndarray'>
(959, 6)
---probability---
Liver : 0.060831
Cyst  : 0.000000
FNH   : 0.093275
CCC   : 0.102737
HCC   : 0.262927
Meta  : 0.480230
